In [ ]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, subprocess
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator   
import nltk
from nltk.corpus import stopwords  
from nltk.stem import PorterStemmer
ps = PorterStemmer()
sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

#df, code = generate_code_from_db(ctx, qgroup=4028)


In [ ]:
def normalizedf(df):
    df.drop( columns=[c for c in df.columns if '__' in c ] , inplace=True )   
    cols = list(df)   
    df['DATA_TYPE']=df['DATA_TYPE'].str.upper()  
    df['CHARACTER_MAXIMUM_LENGTH']=df['CHARACTER_MAXIMUM_LENGTH'].fillna(0).astype(int)
    df['CHARACTER_MAXIMUM_LENGTH']=df['CHARACTER_MAXIMUM_LENGTH'].apply(lambda s: 4000 if s < 0 else s )
    df[cols] = df[cols].astype(str)
    df['HEADER'] = df['COLUMN_NAME'].apply(lambda s: ''.join([' ' + c if ord(c) < 97 else c for c in s ])[1:])
    df=df.loc[:, ['COLUMN_NAME','DATA_TYPE','CHARACTER_MAXIMUM_LENGTH','HEADER' ]]
    return df

In [90]:
sql=" SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME LIKE 'TIC_CLoudServices' ORDER BY ORDINAL_POSITION ASC "
df=sql_todf(sql, connstr)   
df=normalizedf(df)
df

,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,HEADER
0,PK_CloudServices,INT,0,P K _ Cloud Services
1,PK_OrgSubmission,INT,0,P K _ Org Submission
2,ProviderName,NVARCHAR,50,Provider Name
3,ServiceType,NVARCHAR,10,Service Type
4,Service,NVARCHAR,50,Service
5,ServiceOffering,NVARCHAR,200,Service Offering
6,ATODate,DATE,0,A T O Date
7,SubAgency,NVARCHAR,4000,Sub Agency
8,Comments,NVARCHAR,4000,Comments
9,ATOOnFile,VARCHAR,10,A T O On File


In [ ]:

for i, row in df.iterrows():
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        s=f.read() 
    DATA_TYPE=row['DATA_TYPE']
    COLUMN_NAME=row['COLUMN_NAME']
    s=s.replace('{COLUMN_NAME}', COLUMN_NAME)
    if 'INT' in DATA_TYPE:  
        s=s.replace('{Bind_COLUMN_NAME}', COLUMN_NAME+'_Display')
        s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{COLUMN_NAME}" DataValueField="YN" runat="server" />') 
    else:
        s=s.replace('{Bind_COLUMN_NAME}', COLUMN_NAME )
        if 'Date' in COLUMN_NAME:  
            # print("const "+COLUMN_NAME+" = new CBDatePicker({ id: $(`[id$=_"+COLUMN_NAME+"]`).prop('id'), allowNa: true  }); ") 
            print(
            f"""
                IF CAST(@{COLUMN_NAME} AS NVARCHAR(9))='' 
                BEGIN
                    SET @{COLUMN_NAME}=NULL
                END  """ 
            )
            s=s.replace('<%-- --%>', f'<asp:HiddenField ID="{COLUMN_NAME}" runat="server" />') 
        else: 
            s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{COLUMN_NAME}" runat="server" />') 
    # print( s )

In [ ]:
sheet_cols = [
'Sub Agency / Component',
'Initial Authorizing Entity',
'Authorization Type',
'Cloud Service Provider',
'Cloud Service Offering',
'FedRAMP Package ID',
'Agency ATO Date',
'ATO Issuance Date',
'FedRAMP Authorization Date',
'ATO Expiration Date',
'FedRAMP Annual Assessment Date',
'Service Type',
'Service Model Type',
'ATO Letter with Fedramp PMO',
'Comment'
] 

In [ ]:
df=sql_todf("  SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'TIC_CloudServices'  ", connstr)   
df=normalizedf(df) 
df 

In [ ]:

def get_datafield(df):
    txt=[]   
    for i,r in df.iterrows(): 
        COLUMN_NAME=r['COLUMN_NAME'] 
        DATA_TYPE=r['DATA_TYPE']  
        s=f'<telerik:RadTextBox ID="{COLUMN_NAME}" runat="server"/>'
        if 'CHAR' not in DATA_TYPE:
            s=f'<telerik:RadComboBox ID="{COLUMN_NAME}" runat="server"/>'
        txt.append(f'{s}')
    txt='\n'.join(txt)  
    return txt 

def get_gridcols(df,out=False):
    ldf=df.to_dict(orient='records') 
    code,ddl,dv,sel,dtf=[],[],[],[],[]
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        t=f.read() 
        
        for d in ldf: 
            col=d['COLUMN_NAME'] 
            s=t.replace("{COLUMN_NAME}",d['COLUMN_NAME'])  
            if 'INT' in d['DATA_TYPE'].upper():
                s=s.replace('{Bind_COLUMN_NAME}',f"{d['COLUMN_NAME']}_Display")
                s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{col}" DataValueField="YN" runat="server" />')
                ddl.append(f'BindDDL(e, "{col}", "{col}", "{col}")')
                sel.append(f',(SELECT DisplayValue FROM vwPICKLISTS WHERE PK_Picklist={col}) AS {col}') 
            else: 
                s=s.replace('{Bind_COLUMN_NAME}',f"{d['COLUMN_NAME']}")
                s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{col}" runat="server" />')
                dv.append(f'DirectCast(e.Item.FindControl($"{col}"), RadTextBox).Text = drVal("{col}").ToString()')
                sel.append(f',{col}') 
            code.append(f'{s}')     
    code='\n'.join(code) 
    ddl='\n'.join(ddl)
    dv= '\n'.join(dv)
    sel= '\n'.join(sel)[1:]
    with open(f'{ctx.get_dest()}script.aspx', 'w', encoding='UTF-8') as fw:
        fw.write(f'{code} \n\n {ddl} \n\n {dv} \n\n {sel}')
    if out:  print(txt)
    return {'code':code, 'ddl':ddl,'dv':dv,'sel':sel, 'dtf': dtf}
  
 
p = get_gridcols(df, out=False)
p
print ( p['code'] ) 
 

In [ ]:
sql = get_gridcols(df)
print( sql['code'])

In [ ]:
def get_sql(df,out=False):
    parms=[]
    for i,r in df.iterrows():
        s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()}({r['CHARACTER_MAXIMUM_LENGTH']}) = NULL"
        if r['DATA_TYPE'].upper() == 'INT':
            s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()} = NULL" 
        if r['DATA_TYPE'].upper() == 'DATETIME':
            s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()} = NULL" 
        parms.append(f',{s}')

    into=', '.join([c for c in df['COLUMN_NAME'] if 'PK_' not in c])
    vals=', @'.join([c for c in df['COLUMN_NAME'] if 'PK_' not in c]) 
    ins=f'\n\n \t\t \t\t INSERT INTO MFAEncryption ({into})'
    ins=ins+f'\n \t\t \t\t VALUES (@{vals})'

    parms='\n\t\t'.join(parms)[:] 
    update=', '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])  
    ext='\t\t \t\t AND ' + ' AND '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])

    sel,exp=[],[]
    for i,r in df.iterrows():  
        if r['DATA_TYPE'].upper() == 'INT':   
            sel.append(f", {r['COLUMN_NAME']} ")  
            sel.append(f", (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist={r['COLUMN_NAME']}) AS [{r['COLUMN_NAME']}_Display]"  ) 
        elif r['DATA_TYPE'].upper() == 'DATETIME':
            sel.append(f", {r['COLUMN_NAME']} ")
            sel.append(f", FORMAT({r['COLUMN_NAME']}, 'MM/dd/yyyy') AS [{r['COLUMN_NAME']}_Display]"  ) 
        else:
            sel.append(f", {r['COLUMN_NAME']} ")
    sel='\t\t \t\t'+'\n \t\t \t\t'.join(sel)  
 
    return {'parms':parms,'ins':ins, 'update':update, 'sel':sel, 'ext':ext , 'exp':exp  }


df = sql_todf("SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='TIC_CloudServices'", ctx.connstr) 

#  df.columns
d=get_sql(df, out=False)
print( d['sel']  )

In [ ]:
sql_todf("SELECT * FROM vwPicklists WHERE PK_PicklistType IN (76) ", connstr)   

In [ ]:
sql_todf("SELECT * FROM TIC_CloudServices ", connstr)  